In [1]:
import os
import sys
import glob
import time
import pandas as pd
import numpy as np
from itertools import chain

In [2]:
data_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/utf8_data/final_data"
data_list = glob.glob("%s/*_final.txt" % data_dir)
result_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/raw_result/"
print(data_list)
# for test
# data_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/utf8_data/for_test"
# data_list = glob.glob("%s/*.txt" % data_dir)
# result_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/utf8_data/for_test"

['/home/galaxy/project/m6AQTL/data/data_for_website/final_data/utf8_data/final_data/m6A_final.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/utf8_data/final_data/PseudoU_final.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/utf8_data/final_data/m5C_final.txt']


In [3]:
df_list = [pd.read_table(x, sep="\t") for x in data_list]

In [4]:
df_list[0].head()

,Peak,ID,Conservation score,Spectrum width score,Gene_Name|Ensembl_Gene_ID|Gene_Type|Strand|Supported_sample(s)|PubmedID|source|Tissue,RBP_Position|RBP_RBP|RBP_Species|RBP_Database|RBP_Study,splicing_sites_Gene_ID|splicing_sites_Gene_Name|splicing_sites_Gene_Type|splicing_sites_Genomic_location|splicing_sites_Splicing_Site|splicing_sites_Relative_Position,miRNA_Position|miRNA_Strand|miRNA_miRNA-RNA|miRNA_Target_RNA|miRNA_Target_RNA_Type|miRNA_Method|miRNA_Source,clinvar_Position|clinvar_rs_ID|clinvar_Significant|clinvar_Disease(s)|clinvar_Accession|clinvar_Study_name|clinvar_Review_status,GWAS_Position|GWAS_Rs_ID|GWAS_Pubmed_ID|GWAS_Disease/Trait|GWAS_P_value|GWAS_Database|GWAS_Type|GWAS_TagSNP,eQTL_Position|eQTL_Gene_Name|eQTL_RS_ID|eQTL_eQTL_type|eQTL_Pvalue|eQTL_Cell_or_tissue|eQTL_Condition|eQTL_PubMed_ID,meQTL_Position|meQTL_Gene_Name|meQTL_ID|meQTL_Probe_Name|meQTL_RS_ID|meQTL_meQTL_type|meQTL_Pvalue|meQTL_Cell_or_tissue|meQTL_Condition|meQTL_PubMed_ID,pQTL_Position|pQTL_Gene_Name|pQTL_Uniprot_ID|pQTL_RS_ID|pQTL_pQTL_type|pQTL_Pvalue|pQTL_Cell_or_tissue|pQTL_Condition|pQTL_PubMed_ID,sQTL_Position|sQTL_Gene_Name|sQTL_Splicing_sites_Position|sQTL_Splicing_ID|sQTL_Event_type|sQTL_RS_ID|sQTL_sQTL_type|sQTL_Pvalue|sQTL_Cell_or_tissue|sQTL_Condition|sQTL_PubMed_ID,circQTL_Position|circQTL_Gene_Name|circQTL_circRNA_ID|circQTL_RS_ID|circQTL_circQTL_type|circQTL_Pvalue|circQTL_Cell_or_tissue|circQTL_Condition|circQTL_PubMed_ID,COSMIC_Position|COSMIC_RS_ID|COSMIC_Gene_name|COSMIC_Primary_site|COSMIC_Mutation_Description|COSMIC_Mutation_zygosity|COSMIC_FATHMM_prediction|COSMIC_Pubmed_ID
0,chr1:135063-135308,m6A_ID_00001,0.0580,-2.85,AL627309.7|ENSG00000268903|processed_pseudogen...,NaN,NaN,NaN,NaN,NaN,chr1:135202-135203|AL627309.1|rs147502335|cis|...,NaN,NaN,NaN,NaN,NaN
1,chr1:788796-789694,m6A_ID_00002,0.0543,-2.49,AL669831.5|ENSG00000237491|lincRNA|+|GSE114150...,chr1:788886-788907|CSTF2|Homo sapiens|POSTAR2|...,NaN,NaN,NaN,NaN,chr1:788943-788944|FAM87B|rs28692873|cis|1.4E-...,NaN,NaN,NaN,NaN,NaN
2,chr1:791162-791412,m6A_ID_00003,0.0171,-2.83,AL669831.5|ENSG00000237491|lincRNA|+|GSE114150...,chr1:791312-791396|TARDBP|Homo sapiens|POSTAR2...,NaN,NaN,NaN,NaN,chr1:791389-791390|FAM87B|rs28715113|cis|1.07E...,NaN,NaN,NaN,NaN,NaN
3,chr1:817272-817453,m6A_ID_00004,0.0560,-1.96,FAM87B|ENSG00000177757|lincRNA|+|GSE114150:GSM...,NaN,ENSG00000230092|AL669831.4|transcribed_unproce...,NaN,NaN,NaN,chr1:817340-817341|LINC00115|rs3131972|cis|3.2...,NaN,NaN,NaN,NaN,NaN
4,chr1:817964-818276,m6A_ID_00005,0.0286,-2.12,FAM87B|ENSG00000177757|lincRNA|+|GSE114150:GSM...,NaN,ENSG00000177757|FAM87B|lincRNA|chr1:818204|5'G...,NaN,NaN,NaN,chr1:818093-818094|LINC00115|rs2073814|cis|4.8...,NaN,NaN,NaN,NaN,NaN


In [14]:
#### search from QTLs
def main_process():
    df_list = [pd.read_table(x, sep="\t") for x in data_list]
    columns = df_list[0].columns
    mod_col = [0, 1, 2, 3, 4]
    fea_col = [columns[x] for x in [5, 6, 7]]
    id_dict = {"eQTL": [10, 1, 2], "meQTL": [11, 2, 4], "pQTL": [12, 2, 3], "sQTL": [13, 3, 5], "circQTL": [14, 1, 3],
        "GWAS": [9, 1], "clinvar": [8, 1], "COSMIC": [15, 1]}
    #####################################
    for qtl, pos_list in id_dict.items():
#         if qtl == "pQTL":
        if "QTL" not in qtl:
            print(qtl)
            # get qtl & overlapped features dataframe
            df_mod = get_modification(df_list, pos_list[0], mod_col)
            other_names = [x for x in id_dict.keys() if x != qtl]
            df_qtl = get_overlapped_qtl(df_list, qtl, id_dict, other_names, columns)
            df_features = get_overlapped_features(df_list, qtl, columns, fea_col)
            #
            df_result = df_mod.merge(df_qtl, how="outer").merge(df_features, how="outer")
            del df_result["SNP"]
            # add QTL ID
            df_result = assign_id(qtl, id_dict, df_result)
            df_result.to_csv("%s/%s.txt" % (result_dir, qtl), sep="\t", index=False)
# execute
main_process()

GWAS


/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


RBP
splicing
miRNA
Done
clinvar
RBP
splicing
miRNA
Done
COSMIC
RBP
splicing
miRNA
Done


In [10]:
## eQTL & m6A, m5c, u
def merge_peak_info(df, index_mod, index_qtl, columns):
    df_sub = df.iloc[:, index_mod+[index_qtl]]
    mod_names = [columns[x] for x in index_mod]
    peak_col = "#".join(mod_names)
    col_list, merged_list = [], []
    for x in mod_names:
        col_list.append(df_sub[x])
    for x in range(len(col_list[0])):
        merged_list.append("#".join([str(col_list[y][x]) for y in range(len(col_list))]))
    df_sub[peak_col] = merged_list
    df_result = df_sub[[columns[index_qtl], peak_col]]
    peak_title = df_result[peak_col][0].split("#")[1].split("_")[0] + "_" + peak_col
    # In order to distinguish diff RNA modifications, so rename peak title.
    df_result.columns = [columns[index_qtl], peak_title]
    df_result = df_result[~df_result[columns[index_qtl]].isnull()]
#     print(df_result.head())
    return df_result
    
def get_modification(df_list, index_qtl, mod_col):
    mod_array, mod_list, columns = [], [], df_list[0].columns
    for df in df_list:
        mod_list.append(merge_peak_info(df, mod_col, index_qtl, columns))
    df_pre_mod = mod_list[0].merge(mod_list[1], how="outer").merge(mod_list[2], how="outer")
    df_pre_mod = df_pre_mod[df_pre_mod[columns[index_qtl]].astype(str) != "nan"]
    for i, values in df_pre_mod.iterrows():
        qtls = str(values[columns[index_qtl]]).split("@")
        for qtl in qtls:
            each_line = [qtl] + [str(x) for x in values[1:]]
            mod_array.append(each_line)
    df_mod = pd.DataFrame(mod_array, columns=df_pre_mod.columns)
#     print(df_mod.head())
    return df_mod

In [11]:
def get_overlapped_qtl(df_list, query_name, id_dict, other_names, columns):
    query_index = id_dict[query_name][0]
    query_snp = id_dict[query_name][-1]
    query_list = [y for df in df_list for x in df.iloc[:, query_index] for y in str(x).split("@") if (y != np.nan)&(y!="nan")]
    query_col = [x for x in columns if query_name in x][0]
    df = pd.DataFrame()
    df[query_col] = query_list
    df["SNP"] = df.iloc[:, 0].str.split("|").str[query_snp]
    other_index = [id_dict[x][0] for x in other_names]
    other_snp = [id_dict[x][-1] for x in other_names]
    other_col = [y for x in other_names for y in columns if x == y.split("_")[0]]
    other_qtl = []
    for i in other_index:
        other_qtl.append([y for d in df_list for x in d.iloc[:, i] for y in str(x).split("@") if (y != np.nan)&(y!="nan")])
    for i in range(len(other_qtl)):
        prefix = other_col[i].split("_")[0]
        df_other = pd.DataFrame()
        df_other[other_col[i]] = other_qtl[i]
        df_other["SNP"] = df_other.iloc[:, 0].str.split("|").str[other_snp[i]]
        df = df.merge(df_other, how="left")
#     print(df.head())
    df = df[~df[query_col].isnull()]
    return df

In [12]:
def get_overlapped_features(df_list, qtl_name, columns, fea_cols):
    query_col = [x for x in columns if qtl_name == x.split("_")[0]][0]
    result_list = []
    for feature_col in fea_cols:
        print(feature_col.split("_")[0])
        array_1, feature_list = [], []
        for df in df_list:
            feature_list.append(df[[query_col, feature_col]])
        df_1 = pd.concat(feature_list)
        for i, values in df_1.iterrows():
            qtls, features = str(values[0]).split("@"), str(values[-1]).split("@")
            for qtl in qtls:
                for feature in features:
                    each_line = [feature, qtl]
                    array_1.append(each_line)
        df_2 = pd.DataFrame(array_1, columns=[feature_col, query_col])
#         print(df_2.head())
        final_array = []
        for i, values in df_2.iterrows():
            qtl_pos = str(values[query_col]).split("|")[0]
            if qtl_pos.startswith("chr"):
                qtl_right = int(qtl_pos.split(":")[1].split("-")[1])
                if feature_col.startswith("splicing"):
                    if (values[feature_col] != np.nan) and (values[feature_col] != "nan"):
                        feature_pos = int(str(values[feature_col]).split("|")[3].split(":")[1])
                        if abs(qtl_right - feature_pos) < 100:
                            final_array.append(values.tolist())
                else:
                    feature_pos = str(values[feature_col]).split("|")[0]
                    if feature_pos.startswith("chr"):
                        feature_left, feature_right = int(feature_pos.split(":")[1].split("-")[0]), int(feature_pos.split(":")[1].split("-")[1])
                        if (feature_left <= qtl_right <= feature_right):
                            final_array.append(values.tolist())
        df_3 = pd.DataFrame(final_array, columns=[feature_col, query_col])
#         print(df_3.head())
        result_list.append(df_3)
    df_result = result_list[0].merge(result_list[1], how="outer").merge(result_list[2], how="outer")
    print("Done")
    df_result = df_result[~df_result[query_col].isnull()]
    return df_result

In [13]:
def assign_id(qtl_name, id_dict, df_all):
#     if "QTL" in qtl_name:\
    query_col = [x for x in df_all.columns if qtl_name == x.split("_")[0]][0]
    in_list = df_all[query_col].tolist()
#     pos_list = id_dict[qtl_name]
#     gene_index, snp_index = pos_list[1], pos_list[2]
    df = pd.DataFrame()
    df[query_col] = in_list
    df["Position"] = [x.split("|")[0] if str(x).startswith("chr") else "NA" for x in in_list]
#         df["Gene"] = ["NA" if str(x) == "nan" else str(x).split("|")[gene_index] for x in in_list]
#         df["SNP"] = ["NA" if str(x) == "nan" else str(x).split("|")[snp_index] for x in in_list]
    df = df.sort_values(["Position"]).drop_duplicates()
    del df["Position"]  #
    df["ID"] = ["%s_ID_%s" % (qtl_name, str(num).zfill(len(str(len(df))))) for num in range(1, len(df)+1)]
    df_final = df.merge(df_all, on=query_col)
#     print(df_final.head())
#     sys.exit(1)
    return df_final
#     df.to_csv("%s/%s_id.txt" % (result_dir, qtl_name), sep="\t", index=False)